# Inspect AWEI
## Data Collection



In [ ]:
from pathlib import Path
from multiprocessing import Pool
import pandas as pd
from rich import print as printr
from itertools import chain
from omegaconf import OmegaConf, ListConfig
import numpy as np
from pathlib import Path
from functools import partial
from multiprocessing import Pool
from IPython.display import Image, display

from dacbo.instances import create_instance_set

import awei

from awei.utils.loading import read_results
from awei.utils.plotting import dispatch_plots


def find_paths(rundir, rollout_data_fn: str = "rollout_data.json"):
    return list(Path(rundir).glob(f"**/{rollout_data_fn}"))

def find_paths_multi(rundirs: list[str], find_paths_fun: callable = find_paths) -> list[str]:
    printr("Finding paths")
    with Pool() as pool:
        paths = pool.map(find_paths_fun, rundirs)
    # paths = [list(Path(rdir).glob(f"**/{rollout_data_fn}")) for rdir in rundirs]
    paths = chain(*paths)
    printr("Found paths. Loading data")
    return paths




def get_benchmark(experiment_id: str) -> str:
    if experiment_id.startswith("BBOB"):
        benchmark = "BBOB"
    elif experiment_id.startswith("HPOBench"):
        benchmark = "HPOBench_det"
    else:
        raise ValueError
    return benchmark



def preprocess_data(rollout_df: pd.DataFrame, experiment_id: str, all_sawei_variants: bool = False,
                    key_performance: str = "log regret", dimension: int = 8) -> pd.DataFrame:
    
    benchmark = get_benchmark(experiment_id)

    # Define incumbent method and give special name
    policy_inc = "AWEI_0.1 (last) (New Switch)"
    policy_inc = "AWEI_0.1 (last,True,False)"
    # policy_inc = "AWEI (0.1,until_inc_change,0.1,True,False)"  # inc from ablation
    if benchmark.startswith("HPOBench"):
        policy_inc = "AWEI (0.1,last,0.1,True,False)"
    # policy_inc = "AWEI_0.1 (last,True)"
    if all_sawei_variants:
        pi_inc = "\\textbf{SAWEI_0.1 (last)}"
        pi_inc = "AWEI_0.1 (last,True,False)"
        pi_inc = "SAWEI (atolrel0.1,last)"
        pi_inc = "SAWEI (ours)"
    else:
        pi_inc = "SAWEI (ours)"

    # key_performance = "cost"
    key_pi = "policy_name"

    if key_performance == "log regret":
        rollout_df[key_performance] = -rollout_df["reward"]

    rollout_df["policy_name"][rollout_df["policy_name"] == policy_inc] = pi_inc
    # rollout_df["policy_name"] = rollout_df["policy_name"].str.replace("AWEI", "SAWEI", regex=False)

    # rollout_df = rollout_df[(rollout_df[key_pi].str.contains("New Switch")) | (~rollout_df[key_pi].str.startswith("SAWEI")) | (~rollout_df[key_pi].str.startswith("Portfolio"))]

    # FIlter Portfolio Allocation
    # rollout_df = rollout_df[(rollout_df[key_pi] == pi_inc) | (~rollout_df[key_pi].str.startswith("Portfolio"))]

    # Filter SAWEI variants
    if not all_sawei_variants:
        rollout_df = rollout_df[~((rollout_df[key_pi].str.startswith("AWEI")) & (rollout_df[key_pi] != pi_inc))]
    else:
        ...
        # rollout_df["policy_name"] = rollout_df["policy_name"].str.replace(" (New Switch)", "", regex=False)


    col_map = {
        "ubr": "UBR",
        "alpha": r"$\alpha$",
    }

    rollout_df = rollout_df.rename(columns=col_map)

    if dimension and "dimension" in rollout_df:
        rollout_df = rollout_df[rollout_df["dimension"] == dimension]


    replacement_policies = {
        "EI": r"EI ($\alpha=0.5$)",  # r"EI (WEI($\alpha=0.5$))",
        "PI": r"$\alpha=1$)",  # r"WEI($\alpha=1$)",
        "PI (vanilla)": "PI",
        "EI -> PI, switch after 25%": r"EI $\rightarrow$ PI* (25%)",
        "EI -> PI, switch after 50%": r"EI $\rightarrow$ PI* (50%)",
        "EI -> PI, switch after 75%": r"EI $\rightarrow$ PI* (75%)",
        "EI (v) -> PI (v), switch after 25%": r"EI $\rightarrow$ PI (25%)",
        "EI (v) -> PI (v), switch after 50%": r"EI $\rightarrow$ PI (50%)",
        "EI (v) -> PI (v), switch after 75%": r"EI $\rightarrow$ PI (75%)",
        "Explore (w=0)": r"Explore ($\alpha=0$)",  # r"WEI($\alpha=0$) (Explore)",
        "Gutmann-Sobester Pulse": "Gutmann-Sobester Pulse",
        "Linear EI->PI (5 steps)": r"$\alpha=0.5$ $\rightarrow$ $\alpha=1$ (Steps)",  # r"WEI($\alpha=0.5$) $\rightarrow$ WEI($\alpha=1$) (Steps)",
        "Linear PI->EI (5 steps)": r"$\alpha=1$ $\rightarrow$ $\alpha=0.5$ (Steps)",  # r"WEI($\alpha=1$) $\rightarrow$ WEI($\alpha=0.5$) (Steps)",
        "WEI Turn Auto (EI -> Attitude)": "WEI Turn Auto",
        "WEI Turn Up (PI -> Explore)": "WEI Turn Down",
        "WEI Turn Up (EI -> Exploit)": "WEI Turn Up",
        "AWEI_0.1 (last,True)": "AWEI_0.1 (last,True,False)",
        "AWEI_0.1 (last,True,False)": "SAWEI (atolrel0.1,last)",
        "AWEI_0.1 (last,True,True)": "SAWEI (atolrel0.1,last,auto alpha)",
        'AWEI (0.1,last,0.1,True,False,True,False)': 'SAWEI (multiply)',
        "AWEI (0.1,last,0.1,True,False,False,False,True)": "SAWEI (sym-multiply)",
        "AWEI (0.1,last,0.1,False,False,False,False,False)": "SAWEI (modPI)",
    }
    for k, v in replacement_policies.items():
        rollout_df.loc[rollout_df["policy_name"] == k, "policy_name"] = v

    rollout_df.loc[
        rollout_df["log regret"] == -np.inf, "log regret"
    ] = -10000  # Make plotting/ranking possible and meaningful

    print(rollout_df.columns)
    print(rollout_df["policy_name"].unique())
    p = Path(f"figures/{experiment_id}/data.gzip")
    p.parent.mkdir(exist_ok=True, parents=True)
    rollout_df.to_parquet(f"figures/{experiment_id}/data.gzip")
    print("Saved to", f"figures/{experiment_id}/data.gzip")
    return rollout_df


In [ ]:


"""
Experiment ids:

BBOB
BBOB_ablation
HPOBench
"""



rundirs_dict = {
    "BBOB": [
        # # Instance 1
        "/home/benjamin/Dokumente/code/tmp/DAC-BO/AWEI/awei_runs_sync/awei_runs/2023-03-12/18-41-16",
        "/home/benjamin/Dokumente/code/tmp/DAC-BO/AWEI/awei_runs_sync/awei_runs/2023-03-12/19-02-31",
        "/home/benjamin/Dokumente/code/tmp/DAC-BO/AWEI/awei_runs_sync/awei_runs/2023-03-12/20-05-10",
        "/home/benjamin/Dokumente/code/tmp/DAC-BO/AWEI/awei_runs_sync/awei_runs/2023-03-12/22-20-33",
        "/home/benjamin/Dokumente/code/tmp/DAC-BO/AWEI/awei_runs_sync/awei_runs/2023-03-13/00-14-56",
        "/home/benjamin/Dokumente/code/tmp/DAC-BO/AWEI/awei_runs_sync/awei_runs/2023-03-13/01-45-02",
        "/home/benjamin/Dokumente/code/tmp/DAC-BO/AWEI/awei_runs_sync/awei_runs/2023-03-13/12-20-18",
        # Instance 2
        "/home/benjamin/Dokumente/code/tmp/DAC-BO/AWEI/awei_runs_sync/awei_runs/2023-03-13/21-10-52",
        "/home/benjamin/Dokumente/code/tmp/DAC-BO/AWEI/awei_runs_sync/awei_runs/2023-03-13/22-14-44",
        "/home/benjamin/Dokumente/code/tmp/DAC-BO/AWEI/awei_runs_sync/awei_runs/2023-03-14/00-22-01",
        "/home/benjamin/Dokumente/code/tmp/DAC-BO/AWEI/awei_runs_sync/awei_runs/2023-03-14/02-08-51",
        "/home/benjamin/Dokumente/code/tmp/DAC-BO/AWEI/awei_runs_sync/awei_runs/2023-03-14/03-15-18",
        "/home/benjamin/Dokumente/code/tmp/DAC-BO/AWEI/awei_runs_sync/awei_runs/2023-03-14/04-56-19",
        # Instance 3
        "/home/benjamin/Dokumente/code/tmp/DAC-BO/AWEI/awei_runs_sync/awei_runs/BBOB/2023-03-14/15-56-59",
        "/home/benjamin/Dokumente/code/tmp/DAC-BO/AWEI/awei_runs_sync/awei_runs/BBOB/2023-03-14/16-49-18",
        "/home/benjamin/Dokumente/code/tmp/DAC-BO/AWEI/awei_runs_sync/awei_runs/BBOB/2023-03-14/19-05-32",
        "/home/benjamin/Dokumente/code/tmp/DAC-BO/AWEI/awei_runs_sync/awei_runs/BBOB/2023-03-14/20-49-53",
        "/home/benjamin/Dokumente/code/tmp/DAC-BO/AWEI/awei_runs_sync/awei_runs/BBOB/2023-03-14/21-57-55",
        "/home/benjamin/Dokumente/code/tmp/DAC-BO/AWEI/awei_runs_sync/awei_runs/BBOB/2023-03-14/23-39-01",
        # Linear EI to PI / PI to EI, instance 1,2,3
        "/home/benjamin/Dokumente/code/tmp/DAC-BO/AWEI/awei_runs_sync/awei_runs/BBOB/2023-03-22/08-44-29",
        "/home/benjamin/Dokumente/code/tmp/DAC-BO/AWEI/awei_runs_sync/awei_runs/BBOB/2023-03-22/10-46-30",
        # Switch Fix
        "/home/benjamin/Dokumente/code/tmp/DAC-BO/AWEI/awei_runs_sync/awei_runs/BBOB/2023-03-23/14-13-51",
        "/home/benjamin/Dokumente/code/tmp/DAC-BO/AWEI/awei_runs_sync/awei_runs/BBOB/2023-03-23/16-18-45",
        # LCB
        "/home/benjamin/Dokumente/code/tmp/DAC-BO/AWEI/awei_runs/BBOB/2023-04-27/08-27-01",
        # ModPI
        "/home/benjamin/Dokumente/code/tmp/DAC-BO/AWEI/awei_runs/BBOB/2023-04-26/20-28-40",
    ],
    "BBOB_ablation": [
        "/home/benjamin/Dokumente/code/tmp/DAC-BO/AWEI/awei_runs_sync/awei_runs/BBOB/2023-03-16/14-04-07",
        "/home/benjamin/Dokumente/code/tmp/DAC-BO/AWEI/awei_runs_sync/awei_runs/BBOB/2023-03-16/15-39-46",
        "/home/benjamin/Dokumente/code/tmp/DAC-BO/AWEI/awei_runs_sync/awei_runs/BBOB/2023-03-16/17-11-42",
        "/home/benjamin/Dokumente/code/tmp/DAC-BO/AWEI/awei_runs_sync/awei_runs/BBOB/2023-03-16/18-36-07",
        "/home/benjamin/Dokumente/code/tmp/DAC-BO/AWEI/awei_runs_sync/awei_runs/BBOB/2023-03-16/19-58-54",
        "/home/benjamin/Dokumente/code/tmp/DAC-BO/AWEI/awei_runs_sync/awei_runs/BBOB/2023-03-16/21-26-42",
        "/home/benjamin/Dokumente/code/tmp/DAC-BO/AWEI/awei_runs_sync/awei_runs/BBOB/2023-03-16/22-52-15",
        "/home/benjamin/Dokumente/code/tmp/DAC-BO/AWEI/awei_runs_sync/awei_runs/BBOB/2023-03-17/00-18-00",
        "/home/benjamin/Dokumente/code/tmp/DAC-BO/AWEI/awei_runs_sync/awei_runs/BBOB/2023-03-17/01-48-56",
        "/home/benjamin/Dokumente/code/tmp/DAC-BO/AWEI/awei_runs_sync/awei_runs/BBOB/2023-03-17/03-07-44",
        "/home/benjamin/Dokumente/code/tmp/DAC-BO/AWEI/awei_runs_sync/awei_runs/BBOB/2023-03-17/04-38-16",
        "/home/benjamin/Dokumente/code/tmp/DAC-BO/AWEI/awei_runs_sync/awei_runs/BBOB/2023-03-17/06-08-18",
        "/home/benjamin/Dokumente/code/tmp/DAC-BO/AWEI/awei_runs_sync/awei_runs/BBOB/2023-03-17/07-33-43",
        "/home/benjamin/Dokumente/code/tmp/DAC-BO/AWEI/awei_runs_sync/awei_runs/BBOB/2023-03-17/08-53-26",
        "/home/benjamin/Dokumente/code/tmp/DAC-BO/AWEI/awei_runs_sync/awei_runs/BBOB/2023-03-17/10-19-31",
        "/home/benjamin/Dokumente/code/tmp/DAC-BO/AWEI/awei_runs_sync/awei_runs/BBOB/2023-03-19/23-37-57",
        "/home/benjamin/Dokumente/code/tmp/DAC-BO/AWEI/awei_runs_sync/awei_runs/BBOB/2023-03-19/21-53-38",
        "/home/benjamin/Dokumente/code/tmp/DAC-BO/AWEI/awei_runs_sync/awei_runs/BBOB/2023-03-19/20-08-48",
        "/home/benjamin/Dokumente/code/tmp/DAC-BO/AWEI/awei_runs_sync/awei_runs/BBOB/2023-03-19/18-27-52",
        "/home/benjamin/Dokumente/code/tmp/DAC-BO/AWEI/awei_runs_sync/awei_runs/BBOB/2023-03-19/17-12-27",
        "/home/benjamin/Dokumente/code/tmp/DAC-BO/AWEI/awei_runs_sync/awei_runs/BBOB/2023-03-19/15-42-03",
        "/home/benjamin/Dokumente/code/tmp/DAC-BO/AWEI/awei_runs_sync/awei_runs/BBOB/2023-03-19/14-10-22",
        "/home/benjamin/Dokumente/code/tmp/DAC-BO/AWEI/awei_runs_sync/awei_runs/BBOB/2023-03-19/12-44-42",
        "/home/benjamin/Dokumente/code/tmp/DAC-BO/AWEI/awei_runs_sync/awei_runs/BBOB/2023-03-19/11-09-28",
        "/home/benjamin/Dokumente/code/tmp/DAC-BO/AWEI/awei_runs_sync/awei_runs/BBOB/2023-03-19/09-39-30",
        "/home/benjamin/Dokumente/code/tmp/DAC-BO/AWEI/awei_runs_sync/awei_runs/BBOB/2023-03-20/00-54-17",
        "/home/benjamin/Dokumente/code/tmp/DAC-BO/AWEI/awei_runs_sync/awei_runs/BBOB/2023-03-20/02-18-53",
        "/home/benjamin/Dokumente/code/tmp/DAC-BO/AWEI/awei_runs_sync/awei_runs/BBOB/2023-03-20/03-39-29",
        "/home/benjamin/Dokumente/code/tmp/DAC-BO/AWEI/awei_runs_sync/awei_runs/BBOB/2023-03-20/05-28-15",
        "/home/benjamin/Dokumente/code/tmp/DAC-BO/AWEI/awei_runs_sync/awei_runs/BBOB/2023-03-20/06-58-30",
        "/home/benjamin/Dokumente/code/tmp/DAC-BO/AWEI/awei_runs_sync/awei_runs/BBOB/2023-03-20/08-21-03",
        "/home/benjamin/Dokumente/code/tmp/DAC-BO/AWEI/awei_runs_sync/awei_runs/BBOB/2023-03-20/09-39-35",
        "/home/benjamin/Dokumente/code/tmp/DAC-BO/AWEI/awei_runs_sync/awei_runs/BBOB/2023-03-20/10-51-06",
        "/home/benjamin/Dokumente/code/tmp/DAC-BO/AWEI/awei_runs_sync/awei_runs/BBOB/2023-03-20/12-00-27",
        "/home/benjamin/Dokumente/code/tmp/DAC-BO/AWEI/awei_runs_sync/awei_runs/BBOB/2023-03-20/13-24-27",
        "/home/benjamin/Dokumente/code/tmp/DAC-BO/AWEI/awei_runs_sync/awei_runs/BBOB/2023-03-20/15-17-06",
        "/home/benjamin/Dokumente/code/tmp/DAC-BO/AWEI/awei_runs_sync/awei_runs/BBOB/2023-03-20/17-11-04",
        "/home/benjamin/Dokumente/code/tmp/DAC-BO/AWEI/awei_runs_sync/awei_runs/BBOB/2023-03-20/19-05-24",
    ],
    "HPOBench": [
        "/home/benjamin/Dokumente/code/tmp/DAC-BO/AWEI/awei_runs_sync/awei_runs/HPOBench_det/2023-03-20/23-48-15",
        "/home/benjamin/Dokumente/code/tmp/DAC-BO/AWEI/awei_runs_sync/awei_runs/HPOBench_det/2023-03-20/23-32-52",
        "/home/benjamin/Dokumente/code/tmp/DAC-BO/AWEI/awei_runs_sync/awei_runs/HPOBench_det/2023-03-20/23-15-35",
        "/home/benjamin/Dokumente/code/tmp/DAC-BO/AWEI/awei_runs_sync/awei_runs/HPOBench_det/2023-03-20/23-01-31",
        "/home/benjamin/Dokumente/code/tmp/DAC-BO/AWEI/awei_runs_sync/awei_runs/HPOBench_det/2023-03-21/00-04-03",
        "/home/benjamin/Dokumente/code/tmp/DAC-BO/AWEI/awei_runs_sync/awei_runs/HPOBench_det/2023-03-23/11-52-45",
        # More tasks
        "/home/benjamin/Dokumente/code/tmp/DAC-BO/AWEI/awei_runs/HPOBench_det/2023-03-29/16-17-52",
        "/home/benjamin/Dokumente/code/tmp/DAC-BO/AWEI/awei_runs/HPOBench_det/2023-03-29/16-34-48",
        "/home/benjamin/Dokumente/code/tmp/DAC-BO/AWEI/awei_runs/HPOBench_det/2023-03-29/16-56-29",
        "/home/benjamin/Dokumente/code/tmp/DAC-BO/AWEI/awei_runs/HPOBench_det/2023-03-29/22-30-39",
        "/home/benjamin/Dokumente/code/tmp/DAC-BO/AWEI/awei_runs/HPOBench_det/2023-03-29/22-39-58",
        # LCB
        "/home/benjamin/Dokumente/code/tmp/DAC-BO/AWEI/awei_runs/HPOBench_det/2023-04-27/19-47-57",
        # modPI
        "/home/benjamin/Dokumente/code/tmp/DAC-BO/AWEI/awei_runs/HPOBench_det/2023-05-06/09-30-35",
    ]
}

for experiment_id, rundirs in rundirs_dict.items():
    if experiment_id == "HPOBench":
        printr("Experiment ID:", experiment_id)

        rundirs = rundirs_dict[experiment_id]
        paths = find_paths_multi(rundirs)
        rollout_df = read_results(paths)
        printr("Loaded data.")

        rollout_df = rollout_df[~rollout_df["policy_name"].str.startswith("EI -> PI (v),")]  # this one uses a buggy EI

        print(rollout_df["policy_name"].unique())
        print(rollout_df["benchmark"].unique())
        print(rollout_df.columns)

        all_sawei_variants = True
        if "ablation" in experiment_id:
            all_sawei_variants = True

        rollout_df = preprocess_data(rollout_df=rollout_df, experiment_id=experiment_id, all_sawei_variants=all_sawei_variants)


## Alpha BBOB vs HPOBench
Simply load all rundirs for this.

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt
from matplotlib.lines import Line2D

fn_hpob = "/home/benjamin/Dokumente/code/tmp/DAC-BO/AWEI/awei/figures/HPOBench/data.gzip"
fn_bbob = "/home/benjamin/Dokumente/code/tmp/DAC-BO/AWEI/awei/figures/BBOB/data.gzip"

# Read data
df_hpob = pd.read_parquet(fn_hpob)
df_bbob = pd.read_parquet(fn_bbob)

rollout_df = pd.concat([df_hpob, df_bbob])

cmap = sns.color_palette("colorblind", as_cmap=True)
custom_lines = [Line2D([0], [0], color=cmap[0], lw=4),
                Line2D([0], [0], color=cmap[1], lw=4)]

sns.set(font_scale=2)
sns.set_style("whitegrid")

replacement_policies = {
    "AWEI_0.1 (last,True,False)": "SAWEI (ours)",
    "AWEI (0.1,last,0.1,True,False)": "SAWEI (ours)",
}
for k, v in replacement_policies.items():
    rollout_df.loc[rollout_df["policy_name"] == k, "policy_name"] = v
tdf = rollout_df[rollout_df["policy_name"] == "SAWEI (ours)"]
tdf["benchmark"][tdf["benchmark"] == "HPOBench_det"] = "HPOBench"

def norm_steps(df: pd.DataFrame) -> pd.DataFrame:
    _max = df["step"].max()
    _min = df["step"].min()
    df["step (normalized)"] = (df["step"] - _min ) / (_max - _min)
    return df

tdf = tdf.groupby(by="benchmark", group_keys=True).apply(norm_steps)

fig = plt.Figure(figsize=(4, 4))
ax = fig.add_subplot(111)
ax = sns.lineplot(data=tdf, x="step (normalized)", y=r"$\alpha$", hue="benchmark", palette="colorblind", linewidth=3)
ax.set_xlabel("Step (normalized)")
ax.set_ylabel(r"$\alpha$")
ax.set_xlim(0, 1)
ax.set_ylim(0, 1)
ax.set_aspect("equal")
ax.legend(custom_lines, ["BBOB", "HPOBench"], title=None)
fig.set_tight_layout(True)
fig_fn = Path("figures/alpha_bbob_vs_hpobench.png")
fig.savefig(fig_fn, bbox_inches="tight", dpi=300)
plt.show()

## Specs

In [ ]:
import seaborn as sns

###############################################
experiment_id = "HPOBench"
###############################################
benchmark = get_benchmark(experiment_id)
dimension = 8
if benchmark.startswith("HPOBench"):
    dimension = None
    markevery = 3
else:
    markevery = 15

policies = [
    r"EI ($\alpha=0.5$)",  # r"EI (WEI($\alpha=0.5$))",  #    "EI",
    "PI",
    r"Explore ($\alpha=0$)",  # "Explore",
    "LCB",
    r"$\alpha=0.5$ $\rightarrow$ $\alpha=1$ (Steps)",  # r"EI $\rightarrow$ PI* (Linear)",
    r"$\alpha=1$ $\rightarrow$ $\alpha=0.5$ (Steps)",  # r"PI* $\rightarrow$ EI (Linear)",
    r"EI $\rightarrow$ PI (25%)",
    r"EI $\rightarrow$ PI (50%)",
    r"EI $\rightarrow$ PI (75%)",
    "Gutmann-Sobester Pulse",
    "Portfolio Allocation (n=9)",
    "SAWEI (ours)",
    "SAWEI (modPI)"
]
for p in policies:
    print(p)

# Plot Specs
seaborn_markers = [',', '.', 'o', 'v', '^', '<', '>', '8', 's', 'p', '*', 'h', 'H', 'D', 'd', 'P', 'X']
markers = {p: m for p, m in zip(policies, seaborn_markers)}

palette = "colorblind"
if len(policies) == 13:
    palette = sns.color_palette("colorblind")
    ids = [2, 9, 3]
    p2 = [sns.color_palette("Paired")[i] for i in ids]
    palette = {p: c for p, c in zip(policies, list(palette) + p2)}

## Detect Switch

In [ ]:
from pathlib import Path
from functools import partial
from multiprocessing import Pool
from IPython.display import Image, display
import matplotlib.pyplot as plt
import seaborn as sns
import importlib
import awei.adaptive_weighted_ei

importlib.reload(awei.adaptive_weighted_ei)

detect_switch = awei.adaptive_weighted_ei.detect_switch
apply_moving_iqm = awei.adaptive_weighted_ei.apply_moving_iqm


plt.set_loglevel("info")

sns.set_palette("colorblind")
sns.set(font_scale=1.25)
sns.set_style("whitegrid")


def plot_reward_ubr_gradient(
    group, dpi: int = 100, outdir: str = "figures", identifier: str = "", mark_switches: bool = True,
    key_performance: str = "log regret"
):
    # Reward, UBR, UBR gradient per function and policy
    gid, gdf = group
    # gdf = gdf.copy()
    # gdf = gdf[gdf["seed"] == 4]

    hue = "policy_name"

    x = "step"
    y = [key_performance, r"$\alpha$", "UBR"]

    key_ubr_g = r"$\nabla(UBR)$"
    key_ubr_miqm = "UBR (smoothed)"

    ncols = 4

    window_size = 7
    atol_rel = 0.1

    color_ubr = sns.color_palette("colorblind", n_colors=1)[0]
    color_ubr_g = "darkmagenta"
    color_ubr_miqm = color_ubr
    color_switch = "grey"
    alpha_lines = 0.1
    zorder = 10

    X = gdf["step"]
    U = gdf["UBR"].to_numpy()
    miqm = apply_moving_iqm(U=U, window_size=window_size)
    gdf[key_ubr_miqm] = miqm
    miqm_gradient = np.gradient(miqm)
    gdf[key_ubr_g] = miqm_gradient

    G_abs = np.abs(miqm_gradient)
    max_grad = [np.nanmax(G_abs[: i + 1]) for i in range(len(G_abs))]
    switch = np.array([np.isclose(miqm_gradient[i], 0, atol=atol_rel * max_grad[i]) for i in range(len(miqm_gradient))])
    switch[:window_size] = 0  # misleading signal bc of iqm
    switch_ids = np.where(switch)[0]

    ubr_grad_tol_0 = -np.array(max_grad) * atol_rel
    ubr_grad_tol_1 = +np.array(max_grad) * atol_rel

    # print(switch_ids)

    S = [detect_switch(U[: i + 2], atol_rel=atol_rel)[-1] for i in range(len(U) - 2)]
    switch_ids = np.where(S)[0] - 2

    figsize = (10, 4)
    fig = plt.figure(figsize=figsize, dpi=dpi)
    axes = fig.subplots(nrows=1, ncols=ncols, sharex=True, sharey=False)

    # UBR, UBR (smoothed)
    ax = axes[0]
    ax.plot(X, gdf["UBR"], color=color_ubr, label="UBR", alpha=0.4)
    ax.plot(X, gdf[key_ubr_miqm], color=color_ubr_miqm, ls=":", label=key_ubr_miqm)
    ax.set_xlabel("step")
    ax.set_ylabel("UBR")
    ylim = ax.get_ylim()
    ax.vlines(switch_ids, *ylim, color=color_switch, alpha=alpha_lines, zorder=zorder)
    # ax.legend(loc="upper left")

    # gradient
    ax = axes[1]
    ax.fill_between(gdf[x], ubr_grad_tol_0, ubr_grad_tol_1, alpha=0.5)
    ax = sns.lineplot(data=gdf, x=x, y=key_ubr_g, hue=hue, ax=ax, errorbar="se")
    ylim = ax.get_ylim()
    ax.vlines(switch_ids, *ylim, color=color_switch, alpha=alpha_lines, zorder=zorder)
    ax.get_legend().remove()

    # alpha
    ax = axes[2]
    ax = sns.lineplot(data=gdf, x=x, y=r"$\alpha$", hue=hue, ax=ax, errorbar="se")
    ylim = ax.get_ylim()
    ax.vlines(switch_ids, *ylim, color=color_switch, alpha=alpha_lines, zorder=zorder)
    ax.get_legend().remove()

    # log regret
    ax = axes[3]
    ax = sns.lineplot(data=gdf, x=x, y=key_performance, hue=hue, ax=ax, errorbar="se")
    ylim = ax.get_ylim()
    ax.vlines(switch_ids, *ylim, color=color_switch, alpha=alpha_lines, zorder=zorder)
    ax.get_legend().remove()

    # fig.suptitle(f"BBOB Function {gid}")
    fig.set_tight_layout(True)

    if type(gid) == int:
        gid_id = f"{gid:03d}"
    else:
        gid_id = str(gid)

    fn = Path(outdir) / (identifier + gid_id + ".png")
    fn.parent.mkdir(exist_ok=True, parents=True)
    fig.savefig(fn, dpi=300)

    # plt.show()

    return fn


if benchmark == "BBOB":
    group_keys = "fid"
elif benchmark == "HPOBench":
    group_keys = ["model", "task_id"]
else:
    raise NotImplementedError

df = rollout_df

df = df[df["policy_name"] == "SAWEI (ours)"]
df = df[df["seed"] == 6]
groups = list(df.groupby(by=group_keys))
kwargs = dict(identifier=f"{benchmark}/switch/")
# for group in groups:
#     plot_reward_ubr_gradient(group=group, **kwargs)
with Pool() as pool:
    fns = pool.map(partial(plot_reward_ubr_gradient, **kwargs), groups)
fns.sort()
for fn in fns:
    print(fn)
    display(Image(filename=fn))


# plot_reward_ubr_gradient(group=groups[3])

## Log Regret, UBR, Alpha

In [ ]:
from pathlib import Path
from functools import partial
from multiprocessing import Pool
from IPython.display import Image, display
import matplotlib.pyplot as plt
import seaborn as sns
import importlib
from IPython.display import IFrame

import awei.adaptive_weighted_ei

importlib.reload(awei.adaptive_weighted_ei)

detect_switch = awei.adaptive_weighted_ei.detect_switch
apply_moving_iqm = awei.adaptive_weighted_ei.apply_moving_iqm


plt.set_loglevel("info")

sns.set_palette("colorblind")
sns.set(font_scale=1.35)
sns.set_style("whitegrid")


def plot_reward_ubr_gradient(
    group, dpi: int = 100, outdir: str = "figures", identifier: str = "", mark_switches: bool = False,
    key_performance: str = "log regret", key_pi: str = "policy_name"
):
    # Reward, UBR, UBR gradient per function and policy
    gid, gdf = group
    # gdf = gdf.copy()
    # gdf = gdf[gdf["seed"] == 4]

    hue = "policy_name"

    x = "step"
    y = ["reward", "ubr", "ubr_gradient", "ubr-opt"]

    alpha_col = gdf.columns[["alpha" in c for c in gdf.columns].index(True)]

    y = [key_performance, "UBR", alpha_col]

    # gdf["mi7gswitch"] = detect_switch(gdf["ubr"].to_numpy(), 7)

    # y = ["ubr", "mi7gswitch"] #, "U_iqr_grad", "switch", "linefit", "slope"]
    # hue = "seed"
    # hue = None

    ncols = len(y)

    if mark_switches:
        ncols += 1
        order = None

    figsize = (10, 5)
    fig = plt.figure(figsize=figsize, dpi=dpi)
    axes = fig.subplots(nrows=1, ncols=ncols, sharex=True, sharey=False)

    hue_order = policies
    style_order = policies

    # hue_order = None
    # style_order = None

    for j, _y in enumerate(y):
        try:
            ax = axes[j]
            _markers = True
            if _y == y[-1]:
                gdf = gdf[gdf[key_pi] != "Gutmann-Sobester Pulse"]
                # _markers = None

            if _y == key_performance:
                ymin, ymax = gdf[_y].min(), gdf[_y].max()
                gdf.loc[gdf[_y] == -np.inf] = -10000
            ax = sns.lineplot(
                data=gdf, x=x, y=_y, hue=hue, ax=ax, errorbar="se", hue_order=hue_order, style="policy_name", style_order=style_order, dashes=False, markers=markers, markevery=markevery, markeredgewidth=0, linewidth=1, palette=palette,# style=None, dashes=True
            )

            # if _y == key_performance:
            #     ax.set_ylim(ymin * 1.02, ymax)
            box = ax.get_position()
            ax.set_position([box.x0, box.y0 + box.height * 0.5, box.width, box.height * 0.5])
            # ax.set_title(_y)
            if j == 1:  # len(y) - 1:
                if hue:
                    # ax.legend(loc='upper right', bbox_to_anchor=(1.1, 1.1))
                    ax.legend(
                        loc="upper center",
                        frameon=False,
                        bbox_to_anchor=(0.5, -0.2),
                        fancybox=True,
                        shadow=False,
                        ncol=4,
                        fontsize=14,
                    )

            else:
                ax.get_legend().remove()
        except:
            pass

    # fig.suptitle(gid)
    # fig.set_tight_layout(True)
    # fig.subplots_adjust(wspace=-1, hspace=0)

    fig.subplots_adjust(wspace=0.75, hspace=0)

    if type(gid) == int:
        gid_id = f"{gid:03d}"
    else:
        gid_id = str(gid)

    fn = Path(outdir) / (identifier + gid_id + ".pdf")
    fn.parent.mkdir(exist_ok=True, parents=True)
    fig.savefig(fn, dpi=300, bbox_inches="tight")

    # plt.show()

    return fn


if benchmark == "BBOB":
    group_keys = "fid"
elif benchmark.startswith("HPOBench"):
    group_keys = ["model", "task_id"]
else:
    raise NotImplementedError

df = pd.read_parquet(f"figures/{experiment_id}/data.gzip")

groups = list(df.groupby(by=group_keys))
kwargs = dict(identifier=f"{benchmark}/alpha/")
with Pool() as pool:
    fns = pool.map(partial(plot_reward_ubr_gradient, **kwargs), groups)
fns.sort()

for fn in fns:
    print(fn)
    if str(fn).endswith(".pdf"):
        continue
        width = 600
        display(IFrame(fn, width=width, height=0.5*width))
    else:
        display(Image(filename=fn))

## Alpha Grid

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt

if benchmark == "BBOB":
    col_template = "F{col_name}"
elif benchmark.startswith("HPOBench"):
    col_template = "{col_name}"
else:
    raise NotImplementedError(str(benchmark))

sns.set_palette("colorblind")
sns.set(font_scale=1.2)
sns.set_style("whitegrid")

rollout_df = pd.read_parquet(f"figures/{experiment_id}/data.gzip")

plot_df = rollout_df[rollout_df["policy_name"].str.startswith("SAWEI")]
y_key =  plot_df.columns[["alpha" in c for c in plot_df.columns].index(True)]

col = "fid" if benchmark == "BBOB" else "model"
row = None if benchmark == "BBOB" else "task_id"
col_wrap = 4
if row is not None:
    col_wrap = None

grid = sns.FacetGrid(data=plot_df, col=col, row=row, sharex=True, sharey=False, col_wrap=col_wrap, legend_out=True)
grid.map_dataframe(sns.lineplot, x="step", y=y_key, hue="policy_name", hue_order=policies, style="policy_name", style_order=policies, palette="husl")
grid.set_titles(col_template=col_template)
grid.set_xlabels(None)
grid.add_legend(
    title=None
)  # , loc='upper center', bbox_to_anchor=(0.5, -0.2), fancybox=True, shadow=False, ncol=5, fontsize="x-small")
fn = Path(f"figures/{benchmark}/alpha/all.pdf")
fn.parent.mkdir(parents=True, exist_ok=True)
grid.savefig(fn, dpi=300, bbox_inches="tight")

## Final Reward (Ranks)

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt
from rich import print as printr
from functools import partial
from awei.utils.plotting import plot_ranks_per_step, calc_ranks, calc_iqm

plt.set_loglevel("info")

sns.set_palette("colorblind")
sns.set(font_scale=1.5)
sns.set_style("whitegrid")

rollout_df = pd.read_parquet(f"figures/{experiment_id}/data.gzip")

if benchmark == "BBOB":
    group_keys = ["fid", "bbob_instance", "dimension"]
elif benchmark.startswith("HPOBench"):
    group_keys = ["model", "task_id"]
else:
    raise NotImplementedError


final_df = rollout_df[rollout_df["step"] == rollout_df["step"].max() & rollout_df["policy_name"].isin(policies)]

# All Steps
# we calc IQM over the seeds
# if we do it per run, then we see the switches
R = rollout_df[rollout_df["policy_name"].isin(policies)].groupby(by=group_keys + ["episode", "step"]).apply(calc_ranks)
plot_ranks_per_step(R, experiment_id=experiment_id, policies=policies, markevery=markevery, palette=palette)


# Final Step
# Calc rank on each run (last step, fid, instance, dim, episode)
R_final = final_df.groupby(by=group_keys + ["episode"]).apply(calc_ranks)
del R_final["policy_name"]

# Calc IQM over single runs
R_final = R_final.groupby("policy_name").apply(calc_iqm, key="rank")
R_final = R_final.sort_values()
printr(R_final)

# Latex Str
lat_str = R_final.to_latex(escape=False, float_format="{:0.3f}".format)
replacements = {"->": "$\\rightarrow$", "_": " ", "%": "\%"} #, "SAWEI ": "SAWEI $\\epsilon=$\\,"}
for k, v in replacements.items():
    lat_str = lat_str.replace(k, v)
printr(lat_str)


sns.set_palette("colorblind")
sns.set(font_scale=0.75)
sns.set_style("whitegrid")

# palette = sns.color_palette("husl", len(policies))
# palette = {p: c for p, c in zip(policies, palette)}

figsize = (3, 4)
R_final = final_df.groupby(by=group_keys + ["episode"]).apply(calc_ranks)
fig = plt.figure(figsize=figsize, dpi=300)
ax = fig.add_subplot(111)
ax = sns.boxplot(data=R_final, x="policy_name", y="rank", palette=palette, order=None, ax=ax)
ax.tick_params(axis='x', labelrotation=90,)
ax.set_xlabel("Method")
ax.set_ylabel("Log Regret")
fig.set_tight_layout(True)
fn = f"figures/rank_comparison_{experiment_id}.pdf"
fig.savefig(fn, dpi=300, bbox_inches="tight")
plt.show()


## Final Reward Boxplot Grid 

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt

rollout_df = pd.read_parquet(f"figures/{experiment_id}/data.gzip")

if benchmark == "BBOB":
    col_template = "F{col_name}"
elif benchmark.startswith("HPOBench"):
    col_template = "{col_name}"
else:
    raise NotImplementedError(str(benchmark))

sns.set_palette("colorblind")
sns.set(font_scale=1.2)
sns.set_style("whitegrid")

final_df = rollout_df[rollout_df["step"] == rollout_df["step"].max()]
final_df.loc[final_df["log regret"] == -10000, "log regret"] = -np.inf
plot_df = final_df
plot_df = plot_df.sort_values(by="policy_name")

col = "fid" if benchmark == "BBOB" else "model"
row = None if benchmark == "BBOB" else "task_id"
col_wrap = 4
if row is not None:
    col_wrap = None

grid = sns.FacetGrid(data=plot_df, col=col, row=row, sharex=True, sharey=False, col_wrap=col_wrap, legend_out=True)
grid.map_dataframe(sns.boxplot, x="step", y="log regret", hue="policy_name", hue_order=policies, dodge=True, palette=palette)
grid.set_titles(col_template=col_template)
grid.set_xlabels(None)
grid.add_legend(
    title=None
)  # , loc='upper center', bbox_to_anchor=(0.5, -0.2), fancybox=True, shadow=False, ncol=5, fontsize="x-small")
fn = Path(f"figures/{benchmark}/log_regret/all.pdf")
fn.parent.mkdir(parents=True, exist_ok=True)
grid.savefig(fn, dpi=300, bbox_inches="tight")

## Final Log Regret ModPI / ours, BBOB/HPOBench. Violinplot

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt
from matplotlib.lines import Line2D

fn_hpob = "/home/benjamin/Dokumente/code/tmp/DAC-BO/AWEI/awei/figures/HPOBench/data.gzip"
fn_bbob = "/home/benjamin/Dokumente/code/tmp/DAC-BO/AWEI/awei/figures/BBOB/data.gzip"

# Read data
df_hpob = pd.read_parquet(fn_hpob)
df_bbob = pd.read_parquet(fn_bbob)

rollout_df = pd.concat([df_hpob, df_bbob])

cmap = sns.color_palette("colorblind", as_cmap=True)
custom_lines = [Line2D([0], [0], color=cmap[0], lw=4),
                Line2D([0], [0], color=cmap[1], lw=4)]

sns.set(font_scale=2)
sns.set_style("whitegrid")

replacement_policies = {
    "AWEI_0.1 (last,True,False)": "SAWEI (ours)",
    "AWEI (0.1,last,0.1,True,False)": "SAWEI (ours)",
}
for k, v in replacement_policies.items():
    rollout_df.loc[rollout_df["policy_name"] == k, "policy_name"] = v
tdf = rollout_df[rollout_df["policy_name"].isin(["SAWEI (ours)", "SAWEI (modPI)"])]
tdf["benchmark"][tdf["benchmark"] == "HPOBench_det"] = "HPOBench"

def norm_steps(df: pd.DataFrame) -> pd.DataFrame:
    _max = df["step"].max()
    _min = df["step"].min()
    df["step (normalized)"] = (df["step"] - _min ) / (_max - _min)
    return df

tdf = tdf.groupby(by="benchmark", group_keys=True).apply(norm_steps)

fig = plt.Figure(figsize=(4, 4))
ax = fig.add_subplot(111)
ax = sns.lineplot(data=tdf, x="step (normalized)", y=r"$\alpha$", hue="benchmark", style="policy_name", palette="colorblind", linewidth=3)
ax.set_xlabel("Step (normalized)")
ax.set_ylabel(r"$\alpha$")
ax.set_xlim(0, 1)
ax.set_ylim(0, 1)
ax.set_aspect("equal")
ax.legend(
    loc="upper left",
    frameon=False,
    bbox_to_anchor=(1.05, 0.95),
    fancybox=True,
    shadow=False,
    ncol=1,
    fontsize=18,
)
# ax.legend(custom_lines, ["BBOB", "HPOBench"], title=None)
# fig.set_tight_layout(True)
fig_fn = Path("figures/alpha_bbob_vs_hpobench_modPI.pdf")
plt.savefig(fig_fn, bbox_inches="tight", dpi=300)
plt.show()
# plt.clf()

ftdf = tdf[tdf["step (normalized)"] == tdf["step (normalized)"].max()]

sns.set(font_scale=1.3)
sns.set_style("whitegrid")

grid = sns.FacetGrid(data=ftdf, col="benchmark", sharey=False, aspect=1.3)
grid.map_dataframe(sns.violinplot, x="policy_name", hue="policy_name", y="log regret", palette="colorblind", cut=0)
grid.set_xlabels("Method")
grid.set_ylabels("Log Regret")
grid.axes[-1][-1].set_yscale("symlog")
grid.axes[-1][-1].set_ylim(None,-0.1)
grid.tight_layout()
fig_fn = Path("figures/logregret_bbob_vs_hpobench_modPI_violin.pdf")
grid.fig.savefig(fig_fn, bbox_inches="tight", dpi=300)

plt.show()

## UBR Switch

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt

sns.set(font_scale=1)
sns.set_style("whitegrid")

experiment_id = "BBOB"
rollout_df = pd.read_parquet(f"figures/{experiment_id}/data.gzip")

tdf = rollout_df[rollout_df["policy_name"].str.startswith(r"EI $\rightarrow$ PI (")]
n_steps = tdf["step"].max()
switches = [0.25 * n_steps, 0.5 * n_steps, 0.75 * n_steps]
print(tdf.columns)

y_key = "UBR"
# y_key = "log regret"

grid = sns.FacetGrid(data=tdf, col="fid", col_wrap=4, sharex=True, sharey=False)
grid.map_dataframe(sns.lineplot, x="step", y=y_key, hue="policy_name", palette=palette, markers=markers, markevery=markevery, linewidth=2)
grid.add_legend()
grid.savefig(f"figures/BBOB/switch_{y_key}.pdf")

## Attitude Inspection

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

sns.set_palette("colorblind")
sns.set(font_scale=1.)
sns.set_style("whitegrid")


def plot_attitude(group, dpi: int = 100, outdir: str = "figures", identifier: str = ""):
    # Reward, UBR, UBR gradient per function and policy
    gid, gdf = group
    gdf = gdf.copy()

    hue = "policy_name"

    gdf["exploiting"] = gdf["pi_term"] > gdf["ei_term"]

    gdf["pi_term-ei_term"] = gdf["pi_term"] - gdf["ei_term"]

    x = "step"
    

    figsize = (6, 4)

    fig = plt.figure(figsize=figsize, dpi=dpi)
    ax = fig.add_subplot(111)

    # y = ["alpha", "pi_term-ei_term", "exploiting"]
    # y = ["ei_term", "pi_pure_term", "pi_mod_term"]

    # gdfm = gdf.melt(id_vars=[x], value_vars=y, var_name='term', value_name='values')
    
    # ax.set_yscale('symlog')
    # ax = sns.lineplot(data=gdfm, x=x, y="values", hue="term", ax=ax)
    # ax.set_ylabel(None)


    gdf["|ei-pi|"] = np.abs(gdf["ei_term"] - gdf["pi_pure_term"])
    gdf["|ei-pi*|"] = np.abs(gdf["ei_term"] - gdf["pi_mod_term"])
    gdf["|ei-pi|-|ei-pi*|"] = gdf["|ei-pi|"] - gdf["|ei-pi*|"]

    # y = ["|ei-pi|", "|ei-pi*|"]
    # gdfm = gdf.melt(id_vars=[x], value_vars=y, var_name='diff', value_name='values')
    # y = "values"
    # hue = "diff"
    # yscale = "symlog"

    y = ["ei_term", "pi_mod_term", "pi_pure_term"]
    gdfm = gdf.melt(id_vars=[x], value_vars=y, var_name='term', value_name='values')
    y = "values"
    hue = "term"
    yscale = "symlog"

    # gdfm = gdf
    # y = "|ei-pi|-|ei-pi*|"
    # hue = None
    # yscale = "linear"

    ax.set_yscale(yscale)
    ax = sns.lineplot(data=gdfm, x=x, y=y, hue=hue, ax=ax)
    ax.set_ylabel(None)

    fig.suptitle(gid)
    fig.set_tight_layout(True)

    if type(gid) == int:
        gid_id = f"{gid:03d}"
    else:
        gid_id = str(gid)

    fn = Path(outdir) / (identifier + gid_id + ".png")
    fn.parent.mkdir(exist_ok=True, parents=True)
    fig.savefig(fn)

    return fn


rollout_df = pd.read_parquet(f"figures/{experiment_id}/data.gzip")

df = rollout_df[rollout_df["policy_name"].str.startswith("SAWEI (ours)")]
# df = df[df["fid"] == 13]
print(df.columns)
# df = rollout_df
if benchmark == "BBOB":
    group_keys = "fid"
    hue = "fid"
else:
    group_keys = ["model", "task_id"]

fns = dispatch_plots(df, plot_attitude, group_keys=group_keys, identifier=f"{benchmark}/search_attitude/inspect_alpha_attitude_")

dpi = 300
figsize = (6, 4)

fig = plt.figure(figsize=figsize, dpi=dpi)
ax = fig.add_subplot(111)
df["ei-pi"] = df["ei_term"] - df["pi_mod_term"]
ax = sns.lineplot(data=df, x="step", y="ei-pi", hue=group_keys, ax=ax)
ax.set_yscale("symlog")
fig.set_tight_layout(True)
identifier=f"{benchmark}/search_attitude/inspect_alpha_attitude_combined"
fn = Path("figures") / (identifier + ".png")
fn.parent.mkdir(exist_ok=True, parents=True)
fig.savefig(fn)


## Ablation

In [ ]:
import matplotlib.pyplot as plt
import pandas as pd
from rich import print as printr

experiment_id = "BBOB_ablation"
benchmark = get_benchmark(experiment_id)

assert benchmark == "BBOB"

key_performance = "log regret"

rollout_df = pd.read_parquet(f"figures/{experiment_id}/data.gzip")
if key_performance == "log regret":
    rollout_df[key_performance] = -rollout_df["reward"]
df = rollout_df[rollout_df["step"] == rollout_df["step"].max()]


def normalize(df: pd.DataFrame) -> pd.DataFrame:
    lr = df["log regret"].to_numpy()
    lr_min = lr.min()
    lr_max = lr.max()
    lr_norm = (lr - lr_min) / (lr_max - lr_min)
    df["log regret (normalized)"] = lr_norm
    return df


def add_hps(df: pd.DataFrame, cont: bool = True) -> pd.DataFrame:
    pi = df["policy_name"].unique()[0]
    parts = pi.split("(")[1]
    parts = parts.strip(")")
    parts = parts.split(",")
    atol_rel = parts[0]
    track_attitude = parts[1]
    delta = parts[2]
    use_pure_PI = parts[3]
    auto_alpha = parts[4]  # Useless, encoded in delta

    if cont:
        if delta == "auto":
            delta = 1
        else:
            delta = float(delta)

        match track_attitude:
            case "last":
                track_attitude = 0
            case "until_inc_change":
                track_attitude = 1
            case "until_last_switch":
                track_attitude = 2
        atol_rel = float(atol_rel)

    df["atol_rel"] = atol_rel
    df["track_attitude"] = track_attitude
    df["delta"] = delta

    return df


# Normalize log regret per group
df = df.groupby(by=["fid", "bbob_instance", "dimension"], group_keys=False).apply(normalize)

# Add HPs from policy_name (better would be to do it in the loading function 🤓)
df = df.groupby(by="policy_name", group_keys=False).apply(add_hps)

printr(df.columns)



In [ ]:
rollout_df = pd.read_parquet(f"figures/{experiment_id}/data.gzip")
len(rollout_df)

### fANOVA

Load previous cell for this

In [ ]:
import importlib

import fanova
importlib.reload(fanova)
import fanova.visualizer
importlib.reload(fanova.visualizer)
import fanova.visualizer
import logging
import itertools as it
import logging
import os
import pickle
import re
from pathlib import Path

import matplotlib.pyplot as plt
import numpy as np
from ConfigSpace.hyperparameters import Hyperparameter, CategoricalHyperparameter, Constant, OrdinalHyperparameter, \
    NumericalHyperparameter
from matplotlib import cm
from mpl_toolkits.mplot3d import Axes3D

import seaborn as sns

sns.set_palette("colorblind")
sns.set(font_scale=2)
sns.set_style("white")

logging.basicConfig(level=logging.INFO)

from ConfigSpace import ConfigurationSpace, Categorical, OrdinalHyperparameter

config_space = ConfigurationSpace()
hps = [
    OrdinalHyperparameter(name="atol_rel", sequence=[0.05, 0.1, 0.5, 1]),
    Categorical(name="track_attitude", items=[0, 1, 2]),  # "last", "until_inc_change", "until_last_switch"]),
    OrdinalHyperparameter(name="delta", sequence=[0.05, 0.1, 0.25])  #, "auto"]),  # "0.05", "0.1", "0.25", "auto"]),
]
config_space.add_hyperparameters(hps)

printr(config_space.get_hyperparameters())

cols = ["atol_rel", "delta", "track_attitude",]

# for c in cols:
#     printr(df[c].unique())

# Remove delta=auto from the ablation, does not fit the story
df = df[df["delta"] != "auto"]
df = df[df["delta"] != 1]
print(df["delta"].unique())
X = df.loc[:, cols]
y = df.loc[:, "log regret (normalized)"].to_numpy()

f = fanova.fANOVA(X, y, config_space = config_space, n_trees=32, bootstrapping=True)

print(f.trees_total_variance)
print(f.V_U_individual)

v = fanova.visualizer.Visualizer(f, f.cs, ".")

dims = (0,1,2)
vlines = [
    [0.05, 0.1, 0.5, 1],
    [0, 1, 2],
    [0.05, 0.1, 0.25, 1],
]
res = f.quantify_importance(dims)
print(res)

importance_dict = f.quantify_importance([0,1,2])    


for k in sorted(list(importance_dict.keys()), key=lambda t: importance_dict[t]['individual importance'], reverse=True):
    print(k, importance_dict[k])


def plot_marginal(self, param, resolution=100, log_scale=None, show=True, incumbents=None, ax=None):
    """
    Creates a plot of marginal of a selected parameter

    Parameters
    ----------
    param: int or str
        Index of chosen parameter in the ConfigSpace (starts with 0)
    resolution: int
        Number of samples to generate from the parameter range as values to predict
    log_scale: boolean
        If log scale is required or not. If no value is given, it is deduced from the ConfigSpace provided
    show: boolean
        whether to call plt.show() to show plot directly as interactive matplotlib-plot
    incumbents: List[Configuration]
        list of ConfigSpace.Configurations that are marked as incumbents
    """
    param, param_name, param_idx = self._get_parameter(param)

    # check if categorical
    if isinstance(param, NumericalHyperparameter):
        # PREPROCESS
        mean, std, grid = self.generate_marginal(param_idx, resolution)
        mean = np.asarray(mean)
        std = np.asarray(std)

        lower_curve = mean - std
        upper_curve = mean + std

        if log_scale is None:
            log_scale = param.log or (np.diff(grid).std() > 0.000001)

        # PLOT
        if log_scale:
            if np.diff(grid).std() > 0.000001:
                self.logger.info("It might be better to plot this parameter '%s' in log-scale.", param_name)
            plt.semilogx(grid, mean, 'b', label='predicted %s' % self._y_label)
        else:
            plt.plot(grid, mean, 'b', label='predicted %s' % self._y_label)
        plt.fill_between(grid, upper_curve, lower_curve, facecolor='red', alpha=0.6, label='std')

        if incumbents is not None:
            if not isinstance(incumbents, list):
                incumbents = [incumbents]
            values = [inc[param_name] for inc in incumbents if param_name in inc and inc[param_name] is not None]
            indices = [(np.abs(np.asarray(grid) - val)).argmin() for val in values]
            if len(indices) > 0:
                plt.scatter(list([grid[idx] for idx in indices]),
                            list([mean[idx] for idx in indices]),
                            label='incumbent', c='black', marker='.', zorder=999)

        plt.xlabel(param_name)
        plt.ylabel(self._y_label)
        plt.grid(True)
        plt.legend()
        plt.tight_layout()

    else:
        # PREPROCESS
        if isinstance(param, CategoricalHyperparameter):
            labels = param.choices
            categorical_size = len(param.choices)
        elif isinstance(param, OrdinalHyperparameter):
            labels = param.sequence
            categorical_size = len(param.sequence)
        elif isinstance(param, Constant):
            labels = str(param)
            categorical_size = 1
        else:
            raise ValueError("Parameter %s of type %s not supported." % (param.name, type(param)))

        indices = np.arange(1, categorical_size + 1, 1)
        mean, std = self.generate_marginal(param_idx)
        min_y = mean[0]
        max_y = mean[0]

        # PLOT
        if ax is None:
            ax = plt.gca()
        b = ax.boxplot(
            [[x] for x in mean], 
            patch_artist=False, 
            showmeans=False, 
            showfliers=False,
            medianprops={
                "color": "darkslateblue", 
                "linewidth": 5
            },
            boxprops={
                # "facecolor": "C0", 
                # "edgecolor": "white",
                "color": "darkmagenta",
                "linewidth": 5,
            },
            # whiskerprops={"color": "C0", "linewidth": 1.5},
            # capprops={"color": "C0", "linewidth": 1.5}
        )
        ax.set_xticks(indices, labels)

        # blow up boxes
        for box, std_ in zip(b["boxes"], std):
            y = box.get_ydata()
            y[2:4] = y[2:4] + std_
            y[0:2] = y[0:2] - std_
            y[4] = y[4] - std_
            box.set_ydata(y)
            min_y = min(min_y, y[0] - std_)
            max_y = max(max_y, y[2] + std_)

        ax.set_ylim([min_y, max_y])

        # _bp = plt.boxplot([[x] for x in mean], patch_artist=True)
        # for median in _bp['medians']: median.set(color ='forestgreen', linewidth = 3, alpha=0.4) 
        # for cap in _bp['caps']: cap.set(color ='#8B008B', linewidth = 2) 

        ax.set_ylabel(self._y_label)
        ax.set_xlabel(param_name)
    if show:
        plt.show()
    else:
        return ax


sns.set_style("whitegrid")

fig = plt.figure(figsize=(len(dims)*6, 4), dpi=300)
axes = fig.subplots(nrows=1, ncols=len(dims), sharey=True)
for i, dim in enumerate(dims):
    ax = axes[i]
    ax = plot_marginal(v, dim, show=False, ax=ax)
    # ax = plt.gca()
    # x = vlines[dim]
    # ymin, ymax = plt.gca().get_ylim()
    # plt.vlines(x, ymin, ymax)
    match cols[dim]:
        case "track_attitude":
            ax.set_xticks([1, 2, 3])  # hack
            ax.set_xticklabels(["last", "inc. change", "last adjust"], fontsize=19)
        case "delta":
            ax.set_xlabel(r"$\Delta \alpha$")
        case "atol_rel":
            ax.set_xlabel(r"$\epsilon$")
    ax.set_ylabel(None)
axes[0].set_ylabel("log regret (normalized)")
fig.set_tight_layout(False)
fn = Path(f"figures/BBOB/HP_importance/combined.pdf")
fn.parent.mkdir(parents=True, exist_ok=True)
fig.savefig(fn, bbox_inches="tight", dpi=300)
plt.show()
    
# v.plot_pairwise_marginal([0,1], resolution=50)
# v.create_all_plots()
# v.create_most_important_pairwise_marginal_plots()

In [ ]:
df.columns

## Compare Ranks HPOBench and BBOB

In [ ]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

sns.set_palette("colorblind")
sns.set(font_scale=1.)
sns.set_style("whitegrid")

fn_hpob = "/home/benjamin/Dokumente/code/tmp/DAC-BO/AWEI/awei/figures/HPOBench/data.gzip"
fn_bbob = "/home/benjamin/Dokumente/code/tmp/DAC-BO/AWEI/awei/figures/BBOB/data.gzip"

# Read data
df_hpob = pd.read_parquet(fn_hpob)
df_bbob = pd.read_parquet(fn_bbob)

# Final stepe
df_hpob_final = df_hpob[df_hpob["step"] == df_hpob["step"].max()]
df_bbob_final = df_bbob[df_bbob["step"] == df_bbob["step"].max()]

# Calc rank on each run (last step, fid, instance, dim, episode) // (last step, model, task id, episode)
R_hpob = df_hpob_final.groupby(by=["model", "task_id", "episode"]).apply(calc_ranks)
R_bbob = df_bbob_final.groupby(by=["fid", "bbob_instance", "dimension", "episode"]).apply(calc_ranks)
# del R_hpob["policy_name"]
# del R_bbob["policy_name"]

R_hpob["benchmark"] = "HPOBench"
R_bbob["benchmark"] = "BBOB"

R = pd.concat([R_hpob, R_bbob])

# Rank comparison
pal = {
    "BBOB": sns.color_palette("colorblind")[0],
    "HPOBench": sns.color_palette("colorblind")[1],
}
fig = plt.figure(figsize=(7, 1.5), dpi=300)
ax = fig.add_subplot(111)
ax = sns.boxplot(data=R, x="policy_name", y="rank", hue="benchmark", palette=pal, ax=ax)
xticklabes = ax.get_xticklabels()
ax.set_xticklabels(xticklabes, rotation=40, ha='right',)
# ax.tick_params(axis='x', labelrotation=45)
ax.set_xlabel(None)
ax.set_ylabel("Rank")
#ax.legend(loc="upper center", bbox_to_anchor=(0.1, -0.9), fancybox=True, shadow=False, ncol=1, title="Benchmark") #, fontsize="xx-small")
ax.legend(loc="upper center", bbox_to_anchor=(1.15, 0.75), fancybox=True, shadow=False, ncol=1, title="Benchmark") #, fontsize="xx-small")
# fig.set_tight_layout(True)
fn = "figures/rank_comparison.pdf"
fig.savefig(fn, dpi=300, bbox_inches="tight")
plt.show()


# palette = sns.color_palette("husl", len(policies))
# palette = {p: c for p, c in zip(policies, palette)}

figsize = (3, 4)
# Rank BOTH
fig = plt.figure(figsize=figsize, dpi=300)
ax = fig.add_subplot(111)
ax = sns.boxplot(data=R, x="policy_name", y="rank", palette=palette, order=policies, ax=ax)  # , showmeans=True, meanline=True, meanprops={"color": "black", "linewidth": 1.5, "linestyle": ":"},)
ax.tick_params(axis='x', labelrotation=90,)
ax.set_xlabel("Method")
ax.set_ylabel("Rank")
fig.set_tight_layout(True)
fn = "figures/rank_comparison_both.pdf"
fig.savefig(fn, dpi=300, bbox_inches="tight")
plt.show()

figsize = (3, 4)
# Rank BBOB
fig = plt.figure(figsize=figsize, dpi=300)
ax = fig.add_subplot(111)
ax = sns.boxplot(data=R_bbob, x="policy_name", y="rank", palette=palette, order=policies, ax=ax)
ax.tick_params(axis='x', labelrotation=90,)
ax.set_xlabel("Method")
ax.set_ylabel("Rank")
fig.set_tight_layout(True)
fn = "figures/rank_comparison_BBOB.pdf"
fig.savefig(fn, dpi=300, bbox_inches="tight")
plt.show()

# Rank HPOBench
fig = plt.figure(figsize=figsize, dpi=300)
ax = fig.add_subplot(111)
ax = sns.boxplot(data=R_hpob, x="policy_name", y="rank", palette=palette, order=policies, ax=ax)
ax.tick_params(axis='x', labelrotation=90,)
ax.set_xlabel("Method")
ax.set_ylabel("Rank")
fig.set_tight_layout(True)
fn = "figures/rank_comparison_HPOBench.pdf"
fig.savefig(fn, dpi=300, bbox_inches="tight")
plt.show()

# print(R_bbob.index)
# del R_final["policy_name"]

# # Calc IQM over single runs
# R_final = R_hpob.groupby("policy_name").apply(calc_iqm, key="rank")
# R_final = R_final.sort_values()
# printr(R_final)

# # Latex Str
# lat_str = R_final.to_latex(escape=False, float_format="{:0.3f}".format)
# replacements = {"->": "$\\rightarrow$", "_": " ", "%": "\%"} #, "SAWEI ": "SAWEI $\\epsilon=$\\,"}
# for k, v in replacements.items():
#     lat_str = lat_str.replace(k, v)
# printr(lat_str)
